In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
type(sc)

pyspark.context.SparkContext

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [4]:
type(spark)

pyspark.sql.session.SparkSession

In [5]:
type(sc)

pyspark.context.SparkContext

# Los Dataframe

In [6]:
#carga de los archivos csv
df_events = spark.read.load("events.csv",
                    format="csv", sep=",", inferSchema="true", header="true")

In [7]:
df_installs=spark.read.load("installs.csv",
                    format="csv", sep=",", inferSchema="true", header="true")

In [8]:
df_installs.describe()

DataFrame[summary: string, application_id: string, ref_type: string, ref_hash: string, click_hash: string, device_countrycode: string, device_brand: string, device_model: string, session_user_agent: string, user_agent: string, event_uuid: string, kind: string, trans_id: string, ip_address: string, device_language: string]

In [9]:
df_events.describe()

DataFrame[summary: string, event_id: string, ref_type: string, ref_hash: string, application_id: string, device_countrycode: string, device_os_version: string, device_brand: string, device_model: string, device_city: string, session_user_agent: string, trans_id: string, user_agent: string, event_uuid: string, carrier: string, kind: string, device_os: string, connection_type: string, ip_address: string, device_language: string]

In [10]:
df_installs.printSchema()

root
 |-- created: timestamp (nullable = true)
 |-- application_id: integer (nullable = true)
 |-- ref_type: long (nullable = true)
 |-- ref_hash: long (nullable = true)
 |-- click_hash: string (nullable = true)
 |-- attributed: boolean (nullable = true)
 |-- implicit: boolean (nullable = true)
 |-- device_countrycode: long (nullable = true)
 |-- device_brand: decimal(18,-1) (nullable = true)
 |-- device_model: decimal(18,-1) (nullable = true)
 |-- session_user_agent: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- event_uuid: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- wifi: boolean (nullable = true)
 |-- trans_id: string (nullable = true)
 |-- ip_address: long (nullable = true)
 |-- device_language: decimal(18,-1) (nullable = true)



In [11]:
df_events.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- event_id: integer (nullable = true)
 |-- ref_type: long (nullable = true)
 |-- ref_hash: long (nullable = true)
 |-- application_id: integer (nullable = true)
 |-- attributed: boolean (nullable = true)
 |-- device_countrycode: long (nullable = true)
 |-- device_os_version: decimal(19,0) (nullable = true)
 |-- device_brand: decimal(18,-1) (nullable = true)
 |-- device_model: double (nullable = true)
 |-- device_city: decimal(19,0) (nullable = true)
 |-- session_user_agent: double (nullable = true)
 |-- trans_id: string (nullable = true)
 |-- user_agent: double (nullable = true)
 |-- event_uuid: string (nullable = true)
 |-- carrier: decimal(18,-1) (nullable = true)
 |-- kind: decimal(18,-1) (nullable = true)
 |-- device_os: decimal(17,-2) (nullable = true)
 |-- wifi: boolean (nullable = true)
 |-- connection_type: string (nullable = true)
 |-- ip_address: long (nullable = true)
 |-- device_language: decimal(19,0) (nullable = true)



In [12]:
# Creamos una vista temporal asociada a cada dataframe(Temporary View)
df_events.createOrReplaceTempView("events")
df_installs.createOrReplaceTempView("installs")

In [13]:
spark.sql('SELECT wifi from installs LIMIT 10').show()

+----+
|wifi|
+----+
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|true|
+----+



# Analisis N°1: El total de aplicaciones que registraron eventos, en el set de datos.

In [27]:
cantApp_events=spark.sql("SELECT COUNT(DISTINCT(application_id)) as total_app_events\
                    FROM events");
cantApp_events.show()

+----------------+
|total_app_events|
+----------------+
|             269|
+----------------+



In [28]:
application_events= spark.sql("SELECT application_id, COUNT(*) as cant_events\
                                FROM events\
                                GROUP BY application_id\
                                ORDER BY cant_events DESC");

application_events.show(5)

+--------------+-----------+
|application_id|cant_events|
+--------------+-----------+
|            66|     325696|
|            64|     259084|
|           145|     252431|
|            63|     181555|
|           103|     137513|
+--------------+-----------+
only showing top 5 rows



# Analisis N°2: El total de aplicaciones que registraron instalaciones, en el set de datos.

In [26]:
cantApp_installs=spark.sql("SELECT COUNT(DISTINCT(application_id)) as total_app_install\
                            FROM installs");
cantApp_installs.show()

+-----------------+
|total_app_install|
+-----------------+
|               31|
+-----------------+



# Analisis N°3: El top 5 de aplicaciones que registraron instalaciones solamente desde wifi, indicando como resultado el identificador de la aplicación y la cantidad de instalaciones.

In [57]:
query='SELECT installs.application_id, COUNT(installs.wifi) as cant_installs_app\
        FROM installs\
        WHERE installs.wifi="true"\
        GROUP BY application_id\
        ORDER BY cant_installs_app DESC\
        LIMIT 5'
top5_installapp_wifi=spark.sql(query);
top5_installapp_wifi.show(10)

+--------------+-----------------+
|application_id|cant_installs_app|
+--------------+-----------------+
|             9|              498|
|            10|              372|
|            16|              280|
|             2|              164|
|            15|               20|
+--------------+-----------------+



# Analisis N°4: Encontrar los 5 dispositivos que realizaron mayor cantidad de eventos, indicando como resultado el identificador del dispositivo y la cantidad de eventos realizados.



In [29]:
query2='SELECT events.ref_hash as id_device, COUNT(*) as cant_events\
        FROM events\
        GROUP BY ref_hash\
        ORDER BY cant_events DESC\
        LIMIT 5'
top5_device_events=spark.sql(query2);
top5_device_events.show(5)

+-------------------+-----------+
|          id_device|cant_events|
+-------------------+-----------+
|7823950631004872496|       2913|
|7298478026707033340|       2822|
|5034957474698180142|       2681|
|5724169280369284055|       2524|
|1706781657278990931|       2300|
+-------------------+-----------+



# Analizar el output del Catalyst Optimizer en este punto. ¿Se realiza algun tipo de Join? En caso afirmativo, ¿Qué tipo de Join físico se realiza?


In [30]:
spark.sql(query2).explain(True)

== Parsed Logical Plan ==
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Sort ['cant_events DESC NULLS LAST], true
      +- 'Aggregate ['ref_hash], ['events.ref_hash AS id_device#2168, 'COUNT(1) AS cant_events#2169]
         +- 'UnresolvedRelation `events`

== Analyzed Logical Plan ==
id_device: bigint, cant_events: bigint
GlobalLimit 5
+- LocalLimit 5
   +- Sort [cant_events#2169L DESC NULLS LAST], true
      +- Aggregate [ref_hash#13L], [ref_hash#13L AS id_device#2168L, count(1) AS cant_events#2169L]
         +- SubqueryAlias events
            +- Relation[date#10,event_id#11,ref_type#12L,ref_hash#13L,application_id#14,attributed#15,device_countrycode#16L,device_os_version#17,device_brand#18,device_model#19,device_city#20,session_user_agent#21,trans_id#22,user_agent#23,event_uuid#24,carrier#25,kind#26,device_os#27,wifi#28,connection_type#29,ip_address#30L,device_language#31] csv

== Optimized Logical Plan ==
GlobalLimit 5
+- LocalLimit 5
   +- Sort [cant_events#2169L DESC NULLS LAST], true
 

__1)__ Primeramente el query2 es parseado y como podemos observar nos muestra __+- 'UnresolvedRelation `events`__ osea relacion no resueltas aun,pues no sabe nada de los atributos en la query2; Luego en __== Analyzed Logical Plan ==__ se utiliza el dataframe original para el analisis, Una vez que el plan esta resuelto Catalyst comienza a optimizar este plan, generando un __== Optimized Logical Plan ==__.
Y por ultimo una vez optimizado el plan Catalyst genera un conjunto de __== Physical Plan ==__ estos seran evaluados con un modelo de costos y el que posee mayor performance sera el plan usado.
***
__2)__ No se realiza ningun join.
